In [34]:
%pip install -qU langchain langchain_openai langchain-core langgraph langchain-community psycopg[binary,pool]==3.2.6 langgraph-checkpoint-postgres langchain-elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.3/906.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.1 MB/s eta 0:00:00


In [4]:
import os
with open("/content/api_key.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey

with open("/content/postgrest.txt") as archivo:
  uribd = archivo.read()

In [30]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage

In [49]:
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore

db_query= ElasticsearchStore(
    es_url="http://0.0.0.0:9200",
    es_user="elastic",
    es_password="claveelastic",
    index_name="lg-proddata",
    embedding=OpenAIEmbeddings())

retriever = db_query.as_retriever()

tool_rag =retriever.as_tool(
        name="busqueda_productos",
        description="Consulta en la informacion de computadoras, y articulos de computo",
    )

In [28]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.prebuilt import create_react_agent

In [ ]:
## Ejecutar solo la primera vez para crear automaticamente las tablas
## [checkpoint_blobs, checkpoint_migrations, checkpoint_writes, checkpoints] en el esquema public
#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)
    checkpointer.setup()

In [64]:
#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # Inicializamos el modelo
    model = ChatOpenAI(model="gpt-4.1-2025-04-14")

    # Agrupamos las herramientas
    tolkit = [tool_rag]

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
            """
            Eres un asistente gentil de ventas de computadoras especializado.
            Utiliza únicamente las herramientas disponibles para responder y brindar infromacion.
            Si no cuentas con una herramienta específica para resolver una pregunta, infórmalo claramente e indica como pueded ayudar.

            Tu objetivo es guiar al cliente de forma amigable, breve y conversacional, como si fueras un amigo experto en tecnología. Sigue estos pasos:

            1. Saluda y pregunta: Da un saludo cálido, pregunta qué busca el cliente y si tiene una idea clara de lo que necesita (ej. laptop para gaming, PC de oficina, accesorios). Si no sabe, sugiere 2-3 opciones populares, priorizando productos con más stock.
            2. Consulta productos: Usa la información de productos segun su necesidad para responder con detalles de productos relevantes (nombre, descripción, precio, stock). Destaca los que tienen mayor disponibilidad.
            3. Envío o tienda: Pregunta si prefiere recoger en tienda o entrega a domicilio (costo adicional de S/20 para compras menores a S/500; gratis si supera S/500). Si no alcanza los S/50, sugiere añadir algo  para obtener envío gratis o confirma si ya lo logró.
            4. Confirmar pedido: Resume el pedido y pregunta si quiere añadir algo más.
            5. Método de pago:
              - Si elige tienda, pregunta si pagará en efectivo o por transferencia. Solicita su nombre y apellido para generar un código de pedido (formato: AAAAMMDD_HHMMSS_NombreApellido, ej. 20250414_153022_JuanPerez).
              - Si elige domicilio, pide una dirección completa y confirma que el pago será por transferencia.
            6. Cierre de compra:
              - Para transferencias, proporciona el número de cuenta 12730317292820 en BankaNet y pide confirmar el pago.
              - Para pago en tienda, entrega el código de pedido.
            7. Estilo: Sé breve, usa un tono entusiasta y natural. Evita tecnicismos a menos que el cliente los mencione. Responde solo lo necesario para avanzar la conversación.

            """),
        ("human", "{messages}"),
        ]
    )
    #inicializamos el agente
    agent_executor = create_react_agent(model, tolkit, checkpointer=checkpointer, prompt=prompt)


    config = {"configurable": {"thread_id": "abc201"}}
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content="pagar en tienda, mi nombre es miguel angel cotrina espinoza")]},
        config,
        stream_mode="values",
    ):
      step["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡De nada, Miguel Angel! 😊 Me alegra haberte ayudado. Si necesitas algún accesorio, soporte o tienes cualquier otra consulta sobre computadoras, aquí estaré para ayudarte. ¡Que disfrutes mucho tu nueva laptop Dell Inspiron 15!

¡Te espero en tienda para el recojo! ¡Buen día! 👋


In [ ]:
# Invocamos de ultimo mensaje
#config = {"configurable": {"thread_id": "abc201"}}
#response = agent_executor.invoke({"messages": [HumanMessage(content="Gracias por la ayuda")]}, config=config)
#response['messages'][-1].pretty_print()